In [ ]:
### this recommender recommends songs bases on aduio features and is limites to one hit wonders 
#from 1900s to 2000s listed on playback fm

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

from random import randint
from time import sleep

from pickle import load

import sys
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.cluster import KMeans

import warnings
warnings.filterwarnings("ignore")

In [2]:
#read in top 100
top100 = pd.read_pickle("top100.pkl") 
#top100

In [4]:
#read in song list with clusters
song_clusters = pd.read_pickle("song_clusters.pkl") 
#song_clusters

,title,artist,Kmeans_6:cluster,uri
0,Family Affair,Mary J. Blige,1,spotify:track:3aw9iWUQ3VrPQltgwvN9Xu
1,Lose Yourself,Eminem,0,spotify:track:5Z01UMMf7V1o0MzF86s6WJ
2,The Joker And The Queen (feat. Taylor Swift),Ed Sheeran,2,spotify:track:6N1K5OVVCopBjGViHs2IvP
3,On My Way (Marry Me),Jennifer Lopez,2,spotify:track:0PjqDT5SzI91kgzGnylyOd
4,Showdown,Electric Light Orchestra,1,spotify:track:5xAZDlzqvI1wflhB4mzYoz
...,...,...,...,...
2266,Dirtee Disco,Dizzee Rascal,0,spotify:track:0APe3BOy7eHbhbsivMqY9Z
2267,What's Not To Love,Dwele,1,spotify:track:3z7RoqiRrSAkFDVp6EFFeK
2268,Giddy On Up,Laura Bell Bundy,1,spotify:track:1OQ0dRegnZE0TVA4mwxhsO
2269,Teach Me How To Dougie,Cali Swag District,1,spotify:track:61LtVmmkGr8P9I2tSPvdpf


In [5]:
# import unsupervised cluster model and the scaler for audio features
normalized = load(open('StandardScaler.pkl', 'rb'))
kmeans = load(open('kmeans_model.pkl', 'rb'))

In [6]:
#rtead file with clientIDs and client secret
secrets_file = open("spotifyIDs.txt","r")
string = secrets_file.read()

In [7]:
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1]

In [8]:
#Initialize SpotiPy
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['cid'],
                                                           client_secret=secrets_dict['cs']))

In [9]:
# ask for song input
track= input("Enter a Song title you like: ")

Enter a Song title you like: More I See


### a user could make mistakes in the user promt, thats why we look at a close match to a title, not just an exact match, fuzzy libary is very useful in this

In [10]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

mat1=[]

#iterate through top100.title to extract the fuzzy ratio (how match the strings match)
for i in top100.title:
    mat1.append(process.extractOne(i, [track], scorer=fuzz.ratio))
#dframe1['matches'] = mat1
  
#dframe1.show()

/Users/test/opt/anaconda3/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [11]:
# look if matching is for at least one value larger than threshold
isintop100=False
treshold=85

for j in  range(len(mat1)):
    if mat1[j][1] > treshold:
        isintop100=True #if fuzz ratio of the strings matches more than 85% than the title is in our list, just has a typo or a space more or so
        
#isintop100

## decide if song is in top 100 or not

In [12]:
if  isintop100: #top100.title.isin([track]).any():  <-- used that earlier
    random_song = randint(0,len(top100.title)) #get a random song number
    print('Your song is a Top 100 track. You might also like: '+ top100.title[random_song]+' from '+ top100.artist[random_song])

    sys.exit('Thanks for choosing us') #ends the script and we dont execute any of the below code
    
#if song is not in the top100 script will continue    

In [13]:
# ask for artist input
artist= input("Please also enter a artist name: ")

Please also enter a artist name: S. Carey


In [14]:
#get spotify audio features of a song thats not in out list

#make api request
#track_api = sp.search(q=track, type='track', limit=1)
track_api = sp.search(q="artist:" + artist + " track:" + track, type="track", limit=1)

#get URI
track_uri=track_api["tracks"]["items"][0]["uri"]
#get audio_features
audio_features=sp.audio_features(track_uri)


In [15]:
#select numeric values
X = pd.DataFrame.from_dict(audio_features)
X_num = X.select_dtypes(include='number')
#X_num

In [16]:
#scale numeric values
X_num_scaled=normalized.transform(X_num)
scaled_df=pd.DataFrame(X_num_scaled,columns=X_num.columns)
#scaled_df

In [17]:
#select columns
scaled_data=scaled_df[['danceability','energy','key','speechiness','acousticness','instrumentalness','liveness','valence','tempo']]

In [18]:
cluster = kmeans.predict(scaled_data)
#cluster[0]

In [19]:
# recommend another song from the cluster we found
same_cluster=song_clusters[song_clusters['Kmeans_6:cluster']==cluster[0]] #filter data for selected clsuter


In [20]:
df=same_cluster.sample() #select a random row of the selected dataframe

In [21]:
print('Based on your song '+ track +' from '+ artist+ ', we can also recommend you '+ df['title'].item() +' from '+df['artist'].item())

Based on your song More I See from S. Carey, we can also recommend you Gold from Kiiara


In [ ]:
#things to improve: 
# -train the model on a lot more and diverse songs than we did
# -find the best classifier which is not necessarily k-emans
# -also incldue thigns like genre type or year of song publication as parameters, so more diverse than just audio features
